In [1]:
# https://www.geeksforgeeks.org/python-program-to-print-the-fibonacci-sequence/?ref=header_search

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import os
import sys
import math
import pickle

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


## Fibonacci Interative Algorithm

In [2]:
def hanoi(ndisks, startPeg=1, endPeg=3): # 2^N
  if ndisks:
    hanoi(ndisks-1, startPeg, 6-startPeg-endPeg)
    # print(f"Move disk {ndisks} from peg {startPeg} to peg {endPeg}")
    hanoi(ndisks-1, 6-startPeg-endPeg, endPeg)

## Frequency Count Method

In [3]:
X_y = []
i=10
for n in range(2,i+1):
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(hanoi)

  lp_wrapper(n)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

Move disk 1 from peg 1 to peg 2
Move disk 2 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 3 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 1 from peg 1 to peg 2
Move disk 2 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 3
Move disk 3 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 1
Move disk 2 from peg 3 to peg 2
Move disk 1 from peg 1 to peg 2
Move disk 4 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 3
Move disk 2 from peg 2 to peg 1
Move disk 1 from peg 3 to peg 1
Move disk 3 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 2
Move disk 2 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 3
Move disk 1 from peg 1 to peg 3
Move disk 2 from peg 1 to peg 2
Move disk 1 from peg 3 to peg 2
Move disk 3 from peg 1 to peg 3
Move disk 1 from peg 2 to peg 1
Move disk 2 from peg 2 to peg 3
Move dis

In [4]:
resultados_com_menor_loss = []
repeat = 5
registros = []
unary_operators_list = ["log", "square", "cube", "sqrt", "round", "exp", "abs"]

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=unary_operators_list
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=unary_operators_list
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=unary_operators_list
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<-1:-7, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           3.875e+06  1.594e+01  y = -0.10314                                  3           3.844e+06  3.993e-03  y = (1.5018 * x₀)                             5           3.823e+06  2.733e-03  y = (1.6886 * (1.5018 * x₀))                  7           2.147e+06  2.884e-01  y = (((x₀ + 1.3391) * x₀) * x₀)               9           8.111e+05  4.868e-01  y = (((x₀ * (1.5018 * x₀)) * 1.6886) * x₀)    11          3.393e+05  

In [5]:
registros_ = registros
file_pickle = 'dados_v2.pck'

In [6]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [7]:
def salvar_dados(dados, key, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)
  else:
    dados_exist = {}
    
  valor_original = dados_exist.get(key)
  if valor_original == None:
    dados_exist.update({key: [dados]})
  else:
    if isinstance(valor_original, list):
      valor_original.append(dados)
    else:
      dados_exist.update({key: [dados]})

  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)
  
caminho_arquivo = file_pickle
novos_dados = lista_melhor_valor[0:3] + [lista_melhor_valor[4]]

salvar_dados(novos_dados, 'tower_hanoi_2^N', caminho_arquivo)

In [8]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = file_pickle
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items():
  print('\u25CF', k)
  for index, item in enumerate(v):
    if index == len(v)-1:
       print('└─', item)
    else:
      print('├─', item)
  print('==========================')

Conteúdo do arquivo pickle:
● binary_search_logx
├─ [6, 1.2914093, 1.37589542429269, log(x0**6) + 5.574131]
└─ [5, 1.5291592, 2.19479656667366, log(x0)**2 + 13.966276]
● boyer_moore_M_plus_N
├─ [9, 1729031400.0, 0.1972007458935316, 752018330.701125*(0.00603869219560025*x0 + 0.00603869219560025*x1 - 1)**4 + 601415.75]
└─ [5, 15754614000.0, 0.3055240605113849, 3621132.2 - 28185.828*x1]
● bubblesort_N^2
├─ [4, 5910.4966, 3.4645076652491285, 1.5253483*x0**2]
└─ [4, 8542.308, 3.303255717418494, 1.5552218*x0**2]
● fibonacci_interative_N
├─ [3, 0.20818244, 3.412549040667958, 2.058653*x0]
└─ [3, 0.20818244, 3.412549040667958, 2.058653*x0]
● fibonacci_recursive_2^N
├─ [15, 111113150.0, 0.37865996607381686, x0**4*(0.0022606994*x0**2 - 0.714783)]
└─ [15, 77371830.0, 0.38657145558895856, x0**2*(x0**2*(0.16452532*x0 - 3.8766947) + 297.9323)]
● heapsort_NlogN
├─ [9, 0.24989581, 1.1851428541960574, 4*x0 - 1.4897935]
└─ [9, 0.24989575, 1.185128414350167, 4*x0 - 1.4897884]
● insertionsort_N^2
├─ [4, 12